In [17]:
import os
import pandas as pd
import ssl
import json

from kafka import KafkaProducer

ssl._create_default_https_context = ssl._create_unverified_context

In [18]:
KAFKA_BOOTSTRAP_SERVERS='localhost:9092'
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

In [19]:
url = "https://cloud-areapd.pd.infn.it:5210/swift/v1/AUTH_d2e941ce4b324467b6b3d467a923a9bc/MAPD_miniDT_stream/data_000019.txt"
df = pd.read_csv(url)

In [20]:
df.dtypes

HEAD             int64
FPGA             int64
TDC_CHANNEL      int64
ORBIT_CNT        int64
BX_COUNTER       int64
TDC_MEAS       float64
dtype: object

In [23]:
jj = df.loc[0:11000].to_json()

In [25]:
producer.send('topic_stream', json.dumps(jj).encode('utf-8'))
producer.flush()

In [17]:
for i in range(0,81):
    i = str(i).zfill(2)
    url = f"https://cloud-areapd.pd.infn.it:5210/swift/v1/AUTH_d2e941ce4b324467b6b3d467a923a9bc/MAPD_miniDT_stream/data_0000{i}.txt"
    df = pd.read_csv(url)
    for j in range(0,df.shape[0]//1000):
        producer.send('a_partitioned_topic', json.dumps(msg).encode('utf-8'))
        producer.flush()
        time.sleep(0.25)
        

NameError: name 'msg' is not defined

In [13]:
producer.send('topic_stream', b'message 1')
producer.flush()

# Loro

In [15]:
import socket
import json
import time
import random

first_names=('John','Andy','Joe','Alice')
last_names=('Johnson','Smith','Jones', 'Millers')

# while 1:
for i in range(20):
    msg = {
        'name': random.choice(first_names),
        'surname': random.choice(last_names),
        'amount': '{:.2f}'.format(random.random()*1000),
        'delta_t': '{:.2f}'.format(random.random()*10),
        'flag': random.choices([0,1], weights=[0.8, 0.2])[0]
    }
    producer.send('topic_stream', json.dumps(msg).encode('utf-8'))
    producer.flush()
    time.sleep(0.25)